In [93]:
from bs4 import BeautifulSoup
import requests
import json
import csv
import datetime
import time


insight_news_info = []
cnt = 0 # 뉴스 number

# csv변환시키는 함수
def convert_to_csv(csv_name, fieldnames, infor):
    try:
        with open(f'./${csv_name}.csv', 'w', newline='') as c:
            fieldnames = fieldnames
            writer = csv.DictWriter(c, fieldnames=fieldnames) 
            writer.writeheader()

            for news in infor:
                print(news)
                writer.writerow(news) 
    except:
        pass

    
# Dict형태로 변환 함수
def dict_infor(**kwargs):
    news_info = []
    context = {}
    for k,v in kwargs.items():
        context[k] = v
    news_info.append(context)
#     print(news_info)
    return news_info



for num in range(1,2):
    time.sleep(5)
    response = requests.get(f'https://www.insight.co.kr/section/money?page={num}')
    soup = BeautifulSoup(response.text, 'html.parser')

    # 메인 페이지 기사 ul
    ul = soup.select_one('body > div.content > div > div.section-wrap > div.section-list > ul')
    # 메인 페이지 기사리스트들 li
    lis = ul.select('li')
    # 현재 카테고리 네임
    category = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > span')
    press = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > a').text
    # 기사 리스트 for문
    # 프리뷰 기사 img, title, date가져옴
    # 현재 카테고리 각각의 리스트에 함께 나오게 하기 위해 단순히 위에서 받은 new_category밑에다 써줌
    
    for li in lis:
        cnt += 1
        preview_img = li.select_one('li > div > a.section-list-article-image')
        preview_img = preview_img['style'].split("(")[1].split(')')[0]
        title = li.select_one('li > div > a.section-list-article-title').text.strip()
        news_category = category.text.split('>')[1].strip()
        date = li.select_one('li > div > span.section-list-article-byline')
        link = li.select_one('li > div > a.section-list-article-title')['href']
        code = link.split('/')[4]
        
        # code로 내용을 가지고 오는대신 다시한번 requests와 parser이용
        news_url = requests.get(link)
        news_url_html = BeautifulSoup(news_url.text, 'html.parser')
        
        # 세부 컨텐츠 안에 주요 내용 selector css         
        content = news_url_html.select_one('body > div.content > div > div.news-container > div.news-wrap > div.news-article > div.news-article-memo')
        # 세부 컨텐츠 안에 이미지, 이미지 여러개이므로 list에 담아줌
        news_image = content.select('img')
        image_url_list = []
        try:
            for image in news_image:
                image_url_list.append(image['src'])

            # 기자 이름이 날짜와 함께 나오는 경우가 있음 \".\" 이 있는 경우는 기자 네임이 있는 경우라 .을 기준으로 나누어서 날짜만 가져옴
            if '·' in date.text:
                date = date.text.strip().split('·')[1]

            else:
                date = date.text.strip()
                print(date)
            date_list = date.replace(' ','-').split('-')
            date_list = ' '.join(date_list).split()
            print(date_list)
            date_list = date_list[0]+'-'+date_list[1]+'-'+date_list[2]+' '+date_list[3]
            print(date_list)
            date_code = datetime.datetime.strptime(date_list,'%Y-%m-%d %H:%M:%S')
    #         print(datetime.datetime.strptime(date_list,'%Y%m%d%H%M%S'))
            print(type(date_code))



            # 이하 똑같은 형식
            insight_news_info = dict_infor(num=cnt, press=press,news_code=code, news_category=news_category, date=date, preview_img=preview_img, title=title, content=content.text,img=image_url_list,ref=link, date_code= date_list)
        except:
            pass
# print(insight_news_info)
convert_to_csv('insight_news_infor',['num', 'press', 'news_code', 'news_category', 'date','preview_img','title', 'content', 'img', 'ref' ],insight_news_info)


2021-07-13 07:13:22
['2021', '07', '13', '07:13:22']
2021-07-13 07:13:22
<class 'datetime.datetime'>
['2021', '06', '29', '16:24:29']
2021-06-29 16:24:29
<class 'datetime.datetime'>
2021-06-02 17:53:23
['2021', '06', '02', '17:53:23']
2021-06-02 17:53:23
<class 'datetime.datetime'>
['2021', '05', '24', '13:07:54']
2021-05-24 13:07:54
<class 'datetime.datetime'>
['2021', '05', '20', '17:20:09']
2021-05-20 17:20:09
<class 'datetime.datetime'>
['2021', '05', '10', '14:33:15']
2021-05-10 14:33:15
<class 'datetime.datetime'>
['2021', '05', '06', '11:09:53']
2021-05-06 11:09:53
<class 'datetime.datetime'>
['2021', '05', '04', '22:06:07']
2021-05-04 22:06:07
<class 'datetime.datetime'>
['2021', '04', '06', '09:59:15']
2021-04-06 09:59:15
<class 'datetime.datetime'>
['2021', '02', '19', '16:09:14']
2021-02-19 16:09:14
<class 'datetime.datetime'>
2021-02-19 09:10:42
['2021', '02', '19', '09:10:42']
2021-02-19 09:10:42
<class 'datetime.datetime'>
['2021', '01', '20', '17:57:46']
2021-01-20 17:57

### 다음 뉴스 기사 스크래핑 조건

0. 해당기사가 json인지 text로 가져올 수 있는지 확인한다 -[0] text/html이다

1. 기사의 카테고리별로 스크래핑을 해온다(정치, 경제, 사회, 국제, 연예(nope), 스포츠(nope), it, 문화). 이때 해당 카테고리가 존재하지 않으면 안해도 된다. 각 카테고리의 전체기사를 스크래핑 한다

2. 날짜는 오늘 날짜로부터 30일이 지난 기사까지만 보여준다(한달전 뉴스까지 화면에 보여준다) . 스크래핑을 항상 현재 날짜부터 10일전까지의 기사를 스크래핑한다.(누적되서 30일전 기사들도 쌓일 예정이다. 이 중 우리는 30일전까지의 기사까지만 보여준다) 


3. 크론탭(celery 이용할 수 있는지 찾아보기)을 이용해 스크래핑을 1시간에 한번씩 해준다 (상황을 보고 판단할 예정)

4. 스크래핑이 도중에 에러가 날 경우에 대비한 예외처리를 해준다.

5. 기사의 세부내용(이미지, 컨텐츠) 자체를 모델에 집어넣은 후 꺼내어 화면에 뿌려줄때 이미지 형식은 이미지 태그를 써서 src에 넣어주는 로직을 짠다

6. 아티클에 넣어야 할 데이터는 
category, press(언론사), potal(포털-다음), code, preview_img, title, date, content(내용이미지, 세부내용), ref(기사 원본 링크) 

#### 1. 스크래핑 순서
https://news.daum.net/breakingnews/society?page=1&regDate=20210715  
society : 사회
politics : 정치
economic : 경제
foreign : 국제
culture : 문화
digital : it
popular : 연예
sports: 스포츠 

사회 -> 정치 -> 경제 -> 국제 -> 문화 -> IT -> 연예 -> 스포츠     

#### 2. 스크래핑 개수와 시간
> 한번 스크래핑을 할때 각 카테고리별로 50개씩 중복되지 않게 스크래핑 한다(날짜기준으로 정렬해서 모델에 넣기)
> 하루에 30분에 한번씩 스크래핑 될 수 있게 한다
#### 3. 예외처리 찾아보기

#### 4. 고려할 점
1. 생각해보면 굳이 며칠전것까지 스크래핑 할 필요는 없는듯 하다. 매일매일 스크래핑 할 텐데 굳이 옛날꺼까지 하려는 이유는 단순히 초반에는 뉴스 데이터가 없을 것이기 때문에 그러는 것 같다

In [37]:
from bs4 import BeautifulSoup
import requests
import json
import csv
from datetime import datetime
import time
import re

category_list = ['society','politics']

for category in category_list:
    for num in range(1,2):
        time.sleep(5)
        print(f'{category} - {num} 페이지 스크래핑 시작 -!')
        date = datetime.today().strftime("%Y%m%d")
        
        response = requests.get(f'https://news.daum.net/breakingnews/{category}?page={num}&regDate={date}')
        soup = BeautifulSoup(response.text, 'html.parser')
        ul = soup.select_one('#mArticle > div.box_etc > ul')
        lis = ul.select('ul > li')
        for li in lis:
            try:
                title = li.select_one('div > strong > a').text
                ref = li.select_one('div > strong > a')['href']
                print(ref)
                press = li.select_one('strong > span').text
                press = press.split('·')
                press = press[0]
                preview_img = li.select_one('a > img')['src']
                news_url = requests.get(ref)
                news_url_html = BeautifulSoup(news_url.text, 'html.parser')
                nav = news_url_html.select_one(f'#kakaoGnb > div > ul >li.{category} > a > span.ir_wa').text

                print(nav)

            except TypeError:
                print('에러발생함')
                pass

society - 1 페이지 스크래핑 시작 -!
https://v.daum.net/v/20210719173951995
사회
https://v.daum.net/v/20210719173947993
사회
https://v.daum.net/v/20210719173929985
사회
https://v.daum.net/v/20210719173914980
사회
https://v.daum.net/v/20210719173913979
사회
https://v.daum.net/v/20210719173910978
사회
https://v.daum.net/v/20210719173909976
사회
https://v.daum.net/v/20210719173904971
사회
https://v.daum.net/v/20210719173901967
사회
https://v.daum.net/v/20210719173900966
사회
https://v.daum.net/v/20210719173838957
사회
https://v.daum.net/v/20210719173833955
사회
https://v.daum.net/v/20210719173828952
사회
https://v.daum.net/v/20210719173802940
사회
https://v.daum.net/v/20210719173758934
사회
politics - 1 페이지 스크래핑 시작 -!
https://v.daum.net/v/20210719173958997
정치
https://v.daum.net/v/20210719173938991
정치
https://v.daum.net/v/20210719173934988
정치
https://v.daum.net/v/20210719173906973
정치
https://v.daum.net/v/20210719173858963
정치
https://v.daum.net/v/20210719173816945
정치
https://v.daum.net/v/20210719173754931
정치
https://v.daum.net/v/

In [2]:
from bs4 import BeautifulSoup
import requests
import json
import csv
from datetime import datetime
import time
import re

category_list = ['society', 'politics', 'economic', 'foreign', 'culture', 'digital', 'popular', 'sports']

for num in range(1,3):
    time.sleep(5)
    print(f'{category} - {num} 페이지 스크래핑 시작 -!')
    date = datetime.today().strftime("%Y%m%d")

    response = requests.get(f'https://news.daum.net/breakingnews/society?page={num}&regDate={date}')
    soup = BeautifulSoup(response.text, 'html.parser')
    ul = soup.select_one('#mArticle > div.box_etc > ul')
    lis = ul.select('ul > li')
    try:
        for li in lis:
            title = li.select_one('div > strong > a').text
            ref = li.select_one('div > strong > a')['href']
            press = li.select_one('strong > span').text
            press = press.split('·')
            press = press[0]
            preview_img = li.select_one('a > img')['src']
            news_url = requests.get(ref)
            news_url_html = BeautifulSoup(news_url.text, 'html.parser')
            

            content = news_url_html.select_one('#harmonyContainer > section')
            lists = content.find_all(["img","p"])
            print()
            print()
            print(lists)
            print()
            print()

    except TypeError:
        pass

society - 1 페이지 스크래핑 시작 -!


[<p class="link_figure"><img alt="[안동=뉴시스] 19일 청송군청 회의실에서 18번째 '새바람 행복버스 현장 간담회'가 진행되고 있다. (사진=경북도 제공) 2021.07.19 *재판매 및 DB 금지" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/newsis/20210719164223017nxpp.jpg" data-org-width="640" dmcf-mid="kskZaDlEBe" dmcf-mtype="image" height="auto" src="https://img1.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/19/newsis/20210719164223017nxpp.jpg" width="658"/></p>, <img alt="[안동=뉴시스] 19일 청송군청 회의실에서 18번째 '새바람 행복버스 현장 간담회'가 진행되고 있다. (사진=경북도 제공) 2021.07.19 *재판매 및 DB 금지" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/newsis/20210719164223017nxpp.jpg" data-org-width="640" dmcf-mid="kskZaDlEBe" dmcf-mtype="image" height="auto" src="https://img1.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/19/newsis/20210719164223017nxpp.jpg" width="658"/>, <p dmcf-pid="klTbdRLh8J" dmcf-ptype="general">[안동=뉴시스] 류상현 기자 = 경북도가 19일 청송군청 회의실



[<p class="link_figure"><img alt="도쿄 입성한 대한민국 대표팀" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/nocut/20210719164203547oaqc.jpg" data-org-width="710" dmcf-mid="kvPD2PnMRZ" dmcf-mtype="image" height="auto" src="https://img2.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/19/nocut/20210719164203547oaqc.jpg" width="658"/></p>, <img alt="도쿄 입성한 대한민국 대표팀" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/nocut/20210719164203547oaqc.jpg" data-org-width="710" dmcf-mid="kvPD2PnMRZ" dmcf-mtype="image" height="auto" src="https://img2.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/19/nocut/20210719164203547oaqc.jpg" width="658"/>, <p dmcf-pid="k6fwDe7O5u" dmcf-ptype="general">올림픽사진공동취재단 goodlh2@cbs.co.kr</p>]




[<p class="link_figure"><img alt="인천지방법원 전경." class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/chosunbiz/20210719164155409ojzo.jpg" data-org-width="647" dm



[<p class="link_figure"><img alt="우승의 기쁨을 나누고 있는 양산시청 여자탁구단 선수들. (양산시청 제공) © 뉴스1" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/NEWS1/20210719164120346zrhv.jpg" data-org-width="640" dmcf-mid="kP3Evg1qJI" dmcf-mtype="image" height="auto" src="https://img4.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/19/NEWS1/20210719164120346zrhv.jpg" width="658"/></p>, <img alt="우승의 기쁨을 나누고 있는 양산시청 여자탁구단 선수들. (양산시청 제공) © 뉴스1" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/NEWS1/20210719164120346zrhv.jpg" data-org-width="640" dmcf-mid="kP3Evg1qJI" dmcf-mtype="image" height="auto" src="https://img4.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/19/NEWS1/20210719164120346zrhv.jpg" width="658"/>, <p dmcf-pid="ktOPEiahZB" dmcf-ptype="general">(경남=뉴스1) 김명규 기자 = 경남 양산시청 여자탁구단이 지난 13일부터 16일까지 강원도 인제군 다목적경기장에서 열린 2021 추계 회장기 실업탁구대회 단체전에서 우승했다.</p>, <p dmcf-pid="kqmfquVEl4" dmcf-ptype="general">양산시청 여자탁

society - 2 페이지 스크래핑 시작 -!


[<p class="link_figure"><img alt="【서울=뉴시스】 노무라입깃해파리의 모습 (제공 = 한국해양과학기술원)" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/newsis/20210719164102001tlzw.jpg" data-org-width="640" dmcf-mid="kLuWi6LlUS" dmcf-mtype="image" height="auto" src="https://img3.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/19/newsis/20210719164102001tlzw.jpg" width="658"/></p>, <img alt="【서울=뉴시스】 노무라입깃해파리의 모습 (제공 = 한국해양과학기술원)" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/newsis/20210719164102001tlzw.jpg" data-org-width="640" dmcf-mid="kLuWi6LlUS" dmcf-mtype="image" height="auto" src="https://img3.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/19/newsis/20210719164102001tlzw.jpg" width="658"/>, <p dmcf-pid="kYRnBSQ1Nx" dmcf-ptype="general"><br/> [울산=뉴시스]유재형 기자 = 울산시는 19일 울산해역에 노무라입깃해파리 주의보가 발령됨에 따라 어업 피해와 해파리 쏘임 사고 예방을 위해 해파리 구제작업을 실시한다고 밝혔다.</p>, <p dmcf-pid="kWjVYDAX7i" dmcf-pty



[<p class="link_figure"><img alt="[AP/뉴시스] 영국 전역서 1월4일부터 접종이 개시될 아스트라제네카-옥스퍼드대 코로나19 백신이 2일 잉글랜드 한 도시의 병원에 배포돼 의료진이 손에 들고 있다. 2021. 1. 3." class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/newsis/20210719164023991mzkh.jpg" data-org-width="640" dmcf-mid="kSjXXI377v" dmcf-mtype="image" height="auto" src="https://img1.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/19/newsis/20210719164023991mzkh.jpg" width="658"/></p>, <img alt="[AP/뉴시스] 영국 전역서 1월4일부터 접종이 개시될 아스트라제네카-옥스퍼드대 코로나19 백신이 2일 잉글랜드 한 도시의 병원에 배포돼 의료진이 손에 들고 있다. 2021. 1. 3." class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/19/newsis/20210719164023991mzkh.jpg" data-org-width="640" dmcf-mid="kSjXXI377v" dmcf-mtype="image" height="auto" src="https://img1.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/19/newsis/20210719164023991mzkh.jpg" width="658"/>, <p dmcf-pid="kKyB8dBn1K" dmcf-ptype="general"><br/> [서울=뉴시스] 안호균 기자 = 코로나19 백신 접종 후 이상

In [ ]:
            content = news_url_html.select_one('#harmonyContainer > section')
#             content = f'"""{content}"""'
            
#             rx = re.compile(r'<img[^>]*src="([^"]+)')
            
#             r = rx.findall(content)
#             print(r)


In [105]:
from bs4 import BeautifulSoup
import requests
import json
import csv
from datetime import datetime
import time
import re

insight_news_info = []

# Dict형태로 변환 함수
def dict_infor(**kwargs):
    news_info = []
    context = {}
    for k,v in kwargs.items():
        context[k] = v
    news_info.append(context)
#     print(news_info)
    return news_info


category_list = ['society', 'politics', 'economic', 'foreign', 'culture', 'digital', 'popular', 'sports']

for cat in category_list:
    for num in range(1,3):
        time.sleep(5)
        print(f'{cat} - {num} 페이지 스크래핑 시작 -!')
        date = datetime.today().strftime("%Y%m%d")
        response = requests.get(f'https://news.daum.net/breakingnews/society?page={num}&regDate={date}')
        soup = BeautifulSoup(response.text, 'html.parser')
        bg = soup.select_one('body')
#         category = bg.select_one('#kakaoGnb > div > ul > li.on > a > span.ir_wa')
        ul = soup.select_one('#mArticle > div.box_etc > ul')
        lis = ul.select('ul > li')
        for li in lis:
            try:
                title = li.select_one('div > strong > a').text
                ref = li.select_one('div > strong > a')['href']
                code = ref.split('/')[4]
#                 print(code)
                press = li.select_one('strong > span').text
                press = press.split('·')
                press = press[0]
                preview_img = li.select_one('a > img')['src']
                news_url = requests.get(ref)
                news_url_html = BeautifulSoup(news_url.text, 'html.parser')
                category = cat
#                 print(category)
                date = news_url_html.select_one('#cSub > div > span > span > span').text
                date = date.strip()
                date_list = date.replace(' ','.').split('.')
                date_list = ' '.join(date_list).split()
                date_list = date_list[0]+'-'+date_list[1]+'-'+date_list[2]+' '+date_list[3]
                date_code = datetime.strptime(date_list,'%Y-%m-%d %H:%M')
                timestamp = time.mktime(date_code.timetuple())
                timestamp = str(timestamp)
#                 print(type(timestamp))
#                 print(timestamp)
                content = news_url_html.select_one('#harmonyContainer > section')
                content = str(content)
                print(type(content))
            except TypeError:
                print('error')
                pass
            insight_news_info = dict_infor( press=press,news_code=code, news_category=category, date=date_code, preview_img=preview_img, title=title, content=content,ref=ref)
            print(insight_news_info)
            print()
            print()

society - 1 페이지 스크래핑 시작 -!
<class 'str'>
[{'press': '뉴시스 ', 'news_code': '20210717111904122', 'news_category': 'society', 'date': datetime.datetime(2021, 7, 17, 11, 19), 'preview_img': 'https://img1.daumcdn.net/thumb/S95x77ht.u/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fnews%2F202107%2F17%2Fnewsis%2F20210717111904446nzef.jpg&scode=media', 'title': '오늘부터 서울 소재 백화점 종사자 대상 코로나19 검사 행정명령', 'content': '<section dmcf-sid="kMXQCvitLk">\n<figure class="figure_frm origin_fig" dmcf-pid="kBSQ1GGOsr" dmcf-ptype="figure">\n<p class="link_figure"><img class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/17/newsis/20210717111904446nzef.jpg" data-org-width="640" dmcf-mid="kQ2BD5Wl0q" dmcf-mtype="image" height="auto" src="https://img4.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/17/newsis/20210717111904446nzef.jpg" width="658"/></p>\n</figure>\n<p dmcf-pid="kwpy8ivp6F" dmcf-ptype="general"><br/> [서울=뉴시스] 홍효식 기자 = 서울시 소재 백화점 운영자·종사자를 대상으로 코로나19 진단검사 행정명령을 실시한 17일

<class 'str'>
[{'press': '연합뉴스 ', 'news_code': '20210717111514080', 'news_category': 'society', 'date': datetime.datetime(2021, 7, 17, 11, 15), 'preview_img': 'https://img1.daumcdn.net/thumb/S95x77ht.u/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fnews%2F202107%2F17%2Fyonhap%2F20210717111514976mmwd.jpg&scode=media', 'title': '충북서 밤새 18명 확진..유흥업소서 감염 확산', 'content': '<section dmcf-sid="kNDOljPkDK">\n<p dmcf-pid="k4EwpMOQtj" dmcf-ptype="general">(청주=연합뉴스) 심규석 기자 = 밤새 충북에서 신종 코로나바이러스 감염증(코로나19) 확진자가 18명 추가 발생했다.</p>\n<figure class="figure_frm origin_fig" dmcf-pid="kS2mU0CDTR" dmcf-ptype="figure">\n<p class="link_figure"><img alt="줄어들지 않는 확진자 [연합뉴스 자료사진]" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/17/yonhap/20210717111514976mmwd.jpg" data-org-width="1024" dmcf-mid="kbz6i5PyI9" dmcf-mtype="image" height="auto" src="https://img2.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/17/yonhap/20210717111514976mmwd.jpg" width="658"/></p>\n<figcaption clas

<class 'str'>
[{'press': '프레시안 ', 'news_code': '20210717111418067', 'news_category': 'society', 'date': datetime.datetime(2021, 7, 17, 11, 14), 'preview_img': 'https://img1.daumcdn.net/thumb/S95x77ht.u/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fnews%2F202107%2F17%2Fpressian%2F20210717111419911rnzq.jpg&scode=media', 'title': "이경재 전주대 교수, '보험,\xa0인문학에 빠지다' 발간", 'content': '<section dmcf-sid="k82YyHDZl7">\n<div dmcf-pid="kuDNI9xjDH" dmcf-ptype="general">\n           [최인 기자(=전주)(chin580@naver.com)]\n           <br/>\n</div>\n<figure class="figure_frm origin_fig" dmcf-pid="k0DR57Koii" dmcf-ptype="figure">\n<p class="link_figure"><img alt="ⓒ전주대" class="thumb_g_article" data-org-src="https://t1.daumcdn.net/news/202107/17/pressian/20210717111419911rnzq.jpg" data-org-width="640" dmcf-mid="kmeVyG4JDB" dmcf-mtype="image" height="auto" src="https://img3.daumcdn.net/thumb/R658x0.q70/?fname=https://t1.daumcdn.net/news/202107/17/pressian/20210717111419911rnzq.jpg" width="658"/></p>\n<figcaption class="txt

KeyboardInterrupt: 

In [3]:
import time
from datetime import datetime, timedelta

print(time.time())
date = time.time()
new_date = datetime.t

1626437426.2404332


In [21]:
from datetime import datetime, timedelta


print(datetime.today())
print(datetime.today().year)
print(datetime.today().month)
print(datetime.today().day)
print(datetime.today().strftime("%Y%m%d"))

# 어제 날짜 가져오기
for num in range(1,10):
    yesterday = datetime.today() - timedelta(num)
    print(yesterday.strftime("%Y%m%d"))


2021-07-15 21:37:40.041790
2021
7
15
20210715
20210714
20210713
20210712
20210711
20210710
20210709
20210708
20210707
20210706


In [29]:
from datetime import datetime

date = datetime.today().strftime("%Y%m%d")
print(date)

20210715


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
import django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings")
django.setup()
from scrapping.models import Press, Category, Article
import datetime

insight_news_info = []
    
# Dict형태로 변환 함수
def dict_infor(**kwargs):
    context = {}
    for k,v in kwargs.items():
        context[k] = v
    
    return context

def parse_insight():
    cnt = 0 # 뉴스 number
    data = {}
    i = 0

    try:
        for num in range(1,20):
            response = requests.get(f'https://www.insight.co.kr/section/health?page={num}')
            soup = BeautifulSoup(response.text, 'html.parser')

            ul = soup.select_one('body > div.content > div > div.section-wrap > div.section-list > ul')
            lis = ul.select('li')
            category = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > span')
            press = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > a').text
        
            for li in lis:
                cnt += 1
                preview_img = li.select_one('li > div > a.section-list-article-image')
                preview_img = preview_img['style'].split("(")[1].split(')')[0]
                print(preview_img)
                title = li.select_one('li > div > a.section-list-article-title').text.strip()
                news_category = category.text.split('>')[1].strip()
                date = li.select_one('li > div > span.section-list-article-byline')
                link = li.select_one('li > div > a.section-list-article-title')['href']
                code = link.split('/')[4]

                news_url = requests.get(link)
                news_url_html = BeautifulSoup(news_url.text, 'html.parser')
                
                content = news_url_html.select_one('body > div.content > div > div.news-container > div.news-wrap > div.news-article > div.news-article-memo')
                news_image = content.select('img')
                image_url_list = []
                
                for image in news_image:
                    image_url_list.append(image['src'])
                
                if '·' in date.text:
                    date = date.text.strip().split('·')[1]

                else:
                    date = date.text.strip()
                date_code = date.replace(' ','-').split('-')
                date_code = ' '.join(date_code).split()
                date_code = date_code[0]+'-'+date_code[1]+'-'+date_code[2]+' '+date_code[3]
                print(date_code)
                # date_code = datetime.datetime.strptime(date_code,'%Y%m%d%H%M%S')
    #         print(datetime.datetime.strptime(date_list,'%Y%m%d%H%M%S'))
                insight_news_info = dict_infor(press=press,news_code=code, news_category=news_category, date=date, date_code=date_code, preview_img=preview_img, title=title, content=content.text,img=image_url_list,ref=link )
                print(insight_news_info)            
                i += 1
                data[i] = insight_news_info
        return data
    except:
        pass
        # except:
        #     print('문제가 있네용')
        #     pass

if __name__=='__main__':

    news_dict = parse_insight()

    try:
        for v in news_dict.values():
            if not Article.objects.filter(title = v['title']):
                Article.objects.create(
                    press=Press.objects.filter(name=v['press']).first(),
                    category=Category.objects.filter(name=v['news_category']).first(),
                    code=v['news_code'],
                    date=v['date'],
                    date_code=v['date_code'],
                    preview_img=v['preview_img'],
                    title=v['title'],
                    img=v['img'],
                    content=v['content'],
                    ref=v['ref']
                    )
    except:
        pass


In [136]:
import re
junk = """<section dmcf-sid="ky3kqcsW2W">
<div class="video_frm" dmcf-pid="kfngmkQXNI" dmcf-ptype="kakaotv">
<div class="layer_vod">
<div class="vod_player">
<iframe allowfullscreen="" class="player_iframe" dmcf-mid="kdQpeH2lQF" dmcf-mtype="video/kakaotv/owner" dmcf-poster-mid="kWxvuXoz1G" frameborder="0" height="370" id="video@kdQpeH2lQF" poster="https://t1.daumcdn.net/news/202107/15/kbs/20210715234928417akwe.jpg" scrolling="no" src="//tv.kakao.com/embed/player/cliplink/420772192?service=daum_news&amp;m_use_inline=true&amp;ios_allow_inline=true&amp;m_prevent_sdk_use=true&amp;wmode=opaque" width="100%"></iframe>
</div>
</div>
</div>
<p dmcf-pid="kPbtWXhsBA" dmcf-ptype="general">[KBS 강릉] [앵커]</p>
<p dmcf-pid="knCQDyHqlS" dmcf-ptype="general">달라진 도로교통법에 따라 전동킥보드를 탈 땐 안전모를 꼭 착용해야 하고 운전면허도 있어야 합니다.</p>
<p dmcf-pid="kRDHUyRwSm" dmcf-ptype="general">그런데 면허도 없이 전동킥보드를 쉽게 빌릴 수 있다고 하는데요.</p>
<p dmcf-pid="knrJhXY2dB" dmcf-ptype="general">무엇이 문제일까요?</p>
<p dmcf-pid="kIR0XB5Pdu" dmcf-ptype="general">이청초 기자가 취재했습니다.</p>
<p dmcf-pid="kEmwlzhpy3" dmcf-ptype="general">[리포트]</p>
<p dmcf-pid="kM7xIBCz1B" dmcf-ptype="general">인도와 차도를 오가고, 역주행까지 하는 전동킥보드.</p>
<p dmcf-pid="kJWlxpHXHs" dmcf-ptype="general">안전모가 아예 없거나 있어도 들고 다니는 사람까지 가지가집니다.</p>
<p dmcf-pid="kq41EXCBIN" dmcf-ptype="general">모두 도로교통법 위반입니다.</p>
<p dmcf-pid="kYPZiE3yIG" dmcf-ptype="general">면허가 있는지 확인해 봤습니다.</p>
<p dmcf-pid="kNghnCuph4" dmcf-ptype="general">[전동킥보드 이용자 A : "(킥보드 타려면 면허 있어야 되는데, 면허 있으세요 혹시?) 아니요."]</p>
<p dmcf-pid="kItuy4gEt5" dmcf-ptype="general">교복을 입은 학생도 눈에 띕니다.</p>
<p dmcf-pid="kycoVHoBlw" dmcf-ptype="general">[전동킥보드 이용자 B : "(이거 등록할 때 면허증을 입력해야 되는 걸로 알고 있거든요?") 아 이거는 등록 안 해도 돼요."]</p>
<p dmcf-pid="kuIifmDbFE" dmcf-ptype="general">어떻게 이런 일이 가능할까?</p>
<p dmcf-pid="kCalMUE6qj" dmcf-ptype="general">공유 전동 킥보드가 많이 모여있는 장소 중 한 곳입니다. </p>
<p dmcf-pid="kDGp2f4LYo" dmcf-ptype="general">어떤 과정을 통해서 면허 없이도 이 킥보드를 이용할 수 있는지 직접 확인해보겠습니다.</p>
<p dmcf-pid="kqSxnlI8yt" dmcf-ptype="general">스마트폰으로 한 업체에 대여 신청했습니다.</p>
<p dmcf-pid="kMpqct84Zz" dmcf-ptype="general">면허 보유를 묻는 화면에, '네'라고 표시하자, 바로 킥보드를 빌릴 수 있었습니다.</p>
<p dmcf-pid="k1LZNJelwq" dmcf-ptype="general">이번엔 촬영 인증 방식을 사용하는 업체의 앱입니다.</p>
<p dmcf-pid="kf6KKhcEVK" dmcf-ptype="general">빈 보도블록을 찍자.</p>
<p dmcf-pid="kzBwZ83VBI" dmcf-ptype="general">'찰칵'</p>
<p dmcf-pid="kBaXgIwIui" dmcf-ptype="general">면허가 등록됐다는 문구가 뜹니다.</p>
<p dmcf-pid="kVlNbXv1MJ" dmcf-ptype="general">계단도.</p>
<p dmcf-pid="kfcDFuF9WB" dmcf-ptype="general">'찰칵'</p>
<p dmcf-pid="keh77vRnPB" dmcf-ptype="general">조각상도.</p>
<p dmcf-pid="kiW5Y4Q7Tz" dmcf-ptype="general">'찰칵'</p>
<p dmcf-pid="kdxltqgpHo" dmcf-ptype="general">결과는 다 마찬가집니다.</p>
<p dmcf-pid="kLvGkbRnIU" dmcf-ptype="general">KBS가 대여업체 6곳의 앱을 시험해 봤더니 4곳의 인증과정이 허술했습니다.</p>
<p dmcf-pid="kkALb3RhyN" dmcf-ptype="general">[한길희/춘천경찰서 경장 : "어떤 분들은 솔직하게 부모님 운전면허 등록해서 탄다. 아니면 면허 있는 다른 지인 친구분이나 이런 걸 빌려서 등록해서 타고 있다."]</p>
<p dmcf-pid="kgeB6qWDda" dmcf-ptype="general">현재 전국 킥보드 운전자의 10% 정도는 여전히 무면허 상태로 운전하는 것으로 경찰은 추정하고 있습니다.</p>
<p dmcf-pid="kwJt83wyHR" dmcf-ptype="general">KBS 뉴스 이청초입니다.</p>
<p dmcf-pid="keRN1fx8HL" dmcf-ptype="general">촬영기자:이장주</p>
<p dmcf-pid="kPVMrWREFM" dmcf-ptype="general">이청초</p>
</section>"""
rx = re.compile(r"<img[^>]*src='([^']+)|r'<p>")
sources = rx.findall(junk)
print(sources)

[]
